<center>
<a href="https://dilbert.com/strip/1995-11-17"><img src="https://assets.amuniversal.com/b1e4d2d09fcd012f2fe600163e41dd5b" width="60%" /></a>
</center>

In [1]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///
%sql PRAGMA foreign_keys = ON

 * sqlite:///
Done.


[]

# 1. Data Manipulation Language 
<a name="dml">
</a>
Dieser Term bezeichnet den Teil von SQL mit dem

- Erzeugen, ändern und löschen von Tupeln
- Abfragen von Relationen, **lesen** von Tupeln

beschäftigt. Bevor wir beginnen müssen wir unsere Datenbank neu anlegen (Sie erinnern sich sicher: In-Memory Datenbank).

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS Kunde(
    Kundennummer INTEGER PRIMARY KEY NOT NULL,
    Vorname TEXT,
    Nachname TEXT CONSTRAINT ImmerNachname NOT NULL,
    Wohnort TEXT,
    PLZ INTEGER CHECK (PLZ BETWEEN 9999 AND 100000),
    Adresse TEXT,
    Geburtstag DATE
);

CREATE TABLE IF NOT EXISTS Bestellung(
    Bestellnummer INTEGER PRIMARY KEY NOT NULL,
    Kunde INTEGER FOREIGN_KEY REFERENCES Kunde(Kundennummer),
    Bestelldatum TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS Artikel(
    Artikelnummer INTEGER NOT NULL,
    Name TEXT,
    Lagermenge INTEGER,
    PRIMARY KEY (Artikelnummer)
);

CREATE TABLE IF NOT EXISTS enthaelt(
    Bestellung INTEGER NOT NULL,
    Artikel INTEGER NOT NULL,
    Menge Integer DEFAULT 1,
    FOREIGN KEY(Artikel) REFERENCES Artikel(Artikelnummer),
    FOREIGN KEY(Bestellung) REFERENCES Bestellung(Bestellnummer),
    PRIMARY KEY(Bestellung, Artikel)
);

 * sqlite:///
Done.
Done.
Done.
Done.


[]

Der Aufbau der Lerneinheit ist wie folgt:


<a href="#dml">1. Data Manipulation Language</a><br/>
<a href="#11">1.1 Einfügen von Daten</a><br/>
<a href="#12">1.2 Ändern von Datensätzen</a><br/>
<a href="#13">1.3 Löschen von Datensätzen</a><br/>
<a href="#2">2. Abfragen von Datenbanken</a><br/>
<a href="#21">2.1 Einfache SELECT Abfragen</a><br/>
<a href="#22">2.2 Umbenennungen</a><br/>
<a href="#23">2.3 Komplexere SELECT Abfragen</a><br/>
<a href="#24">2.4 Subselects</a><br/>
<a href="#3">3. WHERE: Filtern der Ergebnisse</a><br/>
<a href="#31">3.1 Logische Operationen</a><br/>
<a href="#32">3.2 Mengenwertige Abfragen</a><br/>
<a href="#33">3.3 Suche in Texten</a><br/>
<a href="#4">4. Abfragen über mehrere Tabellen: JOINS</a><br/>
<a href="#41">4.1 Die verschiedenen Joins</a><br/>
<a href="#5">5. Mehr SQL</a><br/>
<a href="#51">5.1 Sortieren und Limitieren</a><br/>
<a href="#52">5.2 Gruppieren</a><br/>
<a href="#53">5.3 Transaktionen am Beispiel</a><br/>


<a name="11">
</a>

# 1.1 Einfügen von Daten in eine Datenbank

[![https://www.sqlite.org/images/syntax/insert-stmt.gif](https://www.sqlite.org/images/syntax/insert-stmt.gif)](https://www.sqlite.org/lang_insert.html)

Inzwischen haben wir gelernt wie wir Tabellen anlegen, nun ist die Frage, wie wir Daten dort speichern können. Dazu verwenden wir den `INSERT` Befehl indem wir angeben in welche Tabelle wir einfügen wollen (`INTO` < Tabellenname >), eine Liste der Attribute angeben, deren Werte gleich Folgen (hier: Nachname, Vorname) und dann die Liste der Werte (`VALUES` (...)). Normalerweise können Sie auch mehrere dieser Wertlisten angeben. Das obige Bild zeigt das Selbe aber mit allen Optionen die SQLite unterstützt. Das Meiste ist Konfliktbehandlung im Fall dass ein Tupel nicht eingefügt werden kann, weil z.B. der PK bereits benutzt wird.

```
INSERT INTO <Tabellenname>
  (<Attribut1>, <Attribut2>, ... <AttributN>)
VALUES
  (<AttributWert1>, <AttributWert2>, ... <AttributWertN>)
```

Oben angegeben ist der Standardfall. Sie sehen im Bild darüber noch die Möglichkeit ein `SELECT` Statement anzugeben. Somit können Sie einen neue Tabelle mit vorhandenen Daten aus einer Abfrage befüllen.

<div class=aufgabe>
Warum ist es wohl praktisch, wenn man die Attribute die eingefügt werden vorher angeben kann? Weil man so nicht immer alle Attribute angeben muss!
</div>
<br/>

Wir legen im Folgenden unsere Ersten Datensätze an, um das zu üben:

### Kunden Anlegen:

Als erstes Legen wir den Benutzer Elser an. Wir kennnen von diesem Kunden nur Namen und Vornamen. Dafür erstellen wir einen INSERT Befehl. Als zweites legen wir zwei weitere Benutzer an: Die Herren Schneeberger und Berg in diesem Fall kennen wir den Ort und legen diesen zusätzlich an. Beachten Sie: Wir legen die beiden Benutzer in einem einzigen Ausdruck an.

In [3]:
%%sql
INSERT INTO Kunde(Nachname,Vorname) VALUES ('Elser', 'Benedikt');
INSERT INTO Kunde(Nachname,Vorname,Wohnort) VALUES ('Berg', 'Waldemar','San Francisco'), ('Schneeberger', 'Josef', 'London');

 * sqlite:///
1 rows affected.
2 rows affected.


[]

Ob alles geklappt hat können wir mit dem einfachsten Abfrage Befehl auf die Kundentabelle prüfen (auch hier wieder der SELECT DML Befehl, den wir später erst bearbeiten):

In [7]:
%%sql
SELECT * FROM Kunde;

 * sqlite:///
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,Deggendorf,94469,None,None
2,Waldemar,Berg,San Francisco,None,None,None
3,Josef,Schneeberger,Deggendorf,94469,None,None


Wie Sie Sehen ist alles gut gelaufen. Und ohne dass wir die KundenId angegeben haben ist sie automatisch für uns erstellt worden.

<a name="12">
</a>

## 1.2 Ändern von Daten in der Datenbank
[![https://www.sqlite.org/images/syntax/update-stmt.gif](https://www.sqlite.org/images/syntax/update-stmt.gif)](https://www.sqlite.org/lang_update.html)

```
UPDATE <Tabellenname>
SET
  (<Attribut1>=<Wert>, <Attribut>=<Wert2>, ... <AttributN>=<WertN>)
WHERE
  <Logischer Ausdruck>
```


Wollen wir nun einen Datensatz ändern (UPDATE) ist die Syntax ähnlich wie der INSERT Ausdruck. Nur muss man eben noch angeben welchen Datensatz man ändern will. Daher ist meistens noch eine Selektion als `WHERE` Bedingung im Audruck dabei. Lassen wir das WHERE weg, **ändern wir alle Tupel in der Tabelle**.

Wenn wir also den Kunden Elser und Schneeberger als Ort "Deggendorf" PLZ 94469 setzen wollen machen wir das wie folgt:

In [6]:
%sql UPDATE Kunde SET Wohnort = 'Deggendorf', PLZ = 94469 WHERE Nachname = 'Elser' OR Nachname = 'Schneeberger';

 * sqlite:///
2 rows affected.


[]

<a name="13">
</a>

## 1.3 Löschen von Daten aus der Datenbank
[![https://www.sqlite.org/images/syntax/delete-stmt.gif](https://www.sqlite.org/images/syntax/delete-stmt.gif)](https://www.sqlite.org/lang_delete.html)

```
DELETE FROM <Tabellenname>
WHERE
  <Logischer Ausdruck>
```

Schließlich ist auch die Syntax zum Löschen recht ähnlich: Sie Deklarieren aus welcher Datenbank Sie löschen wollen und filtern welche Datensätze (Tupel) der Tabelle nicht mehr gebraucht werden.:

In [8]:
%sql DELETE from Kunde where Nachname = 'Elser aber dann doch auch nicht';

 * sqlite:///
0 rows affected.


[]

# 2. Abfragen von Datenbanken `SELECT`

[![https://www.sqlite.org/images/syntax/select-stmt.gif](https://www.sqlite.org/images/syntax/select-stmt.gif)](https://www.sqlite.org/lang_select.html)

In der Einführung haben wir gesehen, wie man Tabellen ablegt und diese mit Daten füllt. Wie können wir aber auf diese Daten zugreifen? In unserem Eingangsbeispiel sahen wir, dass die Datenbanksprache SQL sehr dem Englischen entlehnt ist. Daher ist es intuitiv möglich den Zweck eines einfachen Aufrufs zu verstehen. In der Vorlesung hatten wir früh unseren ersten SELECT Befehl gesehen, der eine **Restriktion** und eine **Projektion** enthielt, um damit die Anzahl der Werte und die Anzahl der Attribute einzuschränken. Der SELECT Befehl gibt uns die Inhalte einer Tabelle zurück. Um dies zu tun braucht er zwei Informationen:

- **SELECT Teil:** Was soll zurück gegeben werden? (Projektion)
- **FROM Teil**: Woher sollen die Inhalte kommen?

Optional hinzu kommt meistens noch ein weiterer Teil:

- **WHERE Teil**: Filtern (Restriktion) der Ergebnisse

Und noch weitere **optinale** Teile. Dem obigen Bild nach zu urteilen ist `SELECT`der mächtigste Befehl, dieser Eindruck täuscht auch nicht. Wir fangen einfach an und arbeiten uns in den folgenden Kapiteln vor. Vorher brauchen wir aber auch etwas mehr Daten, um das auch richtig auszureizen. Daher laden wir im folgenden eine größere Datenbank.

In [9]:
%reload_ext sql
%rm bestellungen.db
!wget -qO- https://nextcloud.th-deg.de/s/ZjR7A3QgH7ydHRF/download | sqlite3 bestellungen.db 
%sql sqlite:///bestellungen.db
%sql SELECT COUNT(*) FROM Bestellung;


rm: cannot remove 'bestellungen.db': No such file or directory
   sqlite:///
 * sqlite:///bestellungen.db
Done.


COUNT(*)
100000


## 2.1 Abfragen über eine Tabelle (Einfache SELECT Abfragen)

Wir erinnern uns an den Ersten SQL Befehl zu Beginn des Skripts:

```
SELECT Name,Vorname,Geburtstag FROM Kunden WHERE Ort = 'Plattling';
```

Und leiten also einen grundsätzlichen Aufbau wie folgt ab:

```
SELECT <Attribute> FROM <Tabellenliste> [WHERE <Filter>] [OPTIONEN]
```

Wir haben also zwei fixe Blöcke, die wir uns nun genauer ansehen. Im SELECT Teil werden die Spalten der Tabelle angegeben, die ausgegeben werden sollen. Im FROM Teil der Abfrage spezifizerien Sie, woher die Daten denn herkommen sollen. Dazu geben Sie im einfachsten Fall den Namen einer Tabelle an, z.B. `Kunde`. Komplexere Angaben betrachten wir [später](#joins). Eine wichtige Option sei hier noch angesprochen `LIMIT <Zahl>` gibt nur die `<Zahl>` ersten (je nach Sortierung) Tupel zurück. Wir verwenden das aus Gründen der Übersichtlichkeit häufig. **Probieren Sie es ruhig aus und entfernen Sie den Zusatz um alle Daten zu sehen!**

### 2.1 Der Stern Operator *: Alle Attribute anzeigen

Ihnen ist in den Beispielen sicher bereits aufgefallen, dass ich oft Ausdrücke wie `SELECT * FROM ...` verwende. Der Stern ist eine Abkürzung für alle verfügbaren Attribute aller Tabellen die Sie abfragen. Wenn Sie entweder alle diese auch sehen wollen, oder keine Lust haben die interessanten Attribute aufzulisten können Sie sich das Leben mit dem `*` einfach machen.

Im Text oben stelt *alle attribute aller Tabellen*. Wenn Sie auf alle Attribute einer **bestimmten** Tabelle (die Sie im `FROM`Teil verwenden) zugreifen wollen, können Sie auch den Namen der Tabelle.* verwenden. Also z.B. 

In [10]:
%sql SELECT Kunde.* FROM Kunde LIMIT 10;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,None,None,None,None
2,Waldemar,Berg,San Francisco,None,None,None
3,Josef,Schneeberger,London,None,None,None
4,Fred,Merriweather,Landshut,84036,None,1983-06-12
5,Juan,Kinlaw,Landshut,84036,None,1986-03-23
6,John,Middleton,Landshut,84036,None,1998-01-03
7,Annette,Marshall,Passau,90123,None,1989-08-03
8,Nathan,Barney,Deggendorf,99469,None,1987-08-26
9,Annette,Caplan,Passau,90123,None,1990-08-27
10,Richard,Abdelhamid,Landshut,84036,None,2008-06-01


<a name="22">
</a>

## 2.2 Umbenennen mit AS

Sie können Aliase für Tabellen und Spalen vergeben. Das erhöt die Lesbarkeit Ihrer Abfrage und hilft Ihnen beim Tippen. Wenn Sie im SELECT Teil aliase verwenden benennen Sie gleichzeitig die Ergenbnis Spalte um. 

Beispielsweise übersetzen wir im Folgenden die **Vor und Nachnamen Spalten** ins Englische und sprechen die Kundentabelle als **kd** an:

In [11]:
%sql SELECT kd.Vorname AS 'First Name', kd.Nachname AS 'Last Name' from Kunde as kd LIMIT 2;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


First Name,Last Name
Benedikt,Elser
Waldemar,Berg


<div class="info">
    Sie wollen gleich erfahren, wie man Anfragen auch filtert? Dann springen Sie zum Kapitel <a href="#where">WHERE</a>
    <br>
</div>

<a name="23">
</a>
    
## 2.3 Komplexeres SELECT

Bevor das Ergebnis final ausgegeben wird, können Sie diese im SELECT Block noch einmal anfassen. Übliche Operationen sind z.B., dass man nicht die Werte selbst betrachtet, sondern nur an der Anzahl der Werte interessiert ist (`COUNT` Operator) oder skalare Funktonen (z.B. das maximum aller Werte `MAX`) mit numerischen Daten durchführt. Außerdem kann man bereits dort Text verarbeiten lassen, was nach dem Auslesen der Daten in z.B. EXCEL oft umständlich ist.

Mathematische Operationen können Sie natürlich ebenfalls verwenden, wie die folgende Division durch  1000 um die erste Stelle der Postleitzahl zu errechnen.

In [14]:
%sql SELECT PLZ/10000 as Postleitzahlenbereich,Vorname,Nachname from Kunde LIMIT 5;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Postleitzahlenbereich,Vorname,Nachname
None,Benedikt,Elser
None,Waldemar,Berg
None,Josef,Schneeberger
8,Fred,Merriweather
8,Juan,Kinlaw


### 2.3.1 Funktionen
SQL bietet [einige Aggregatsfunktionen](https://sqlite.org/lang_aggfunc.html) an, die Sie Attribute Ihres Ergebnisses anwenden können. Populär sind z.B. `MAX, MIN, COUNT, SUM, AVG`. Unterschiedliche Datenbanken bieten verschiedene weitere Funktionen, dafür laden Sie aber meistens bei der [Datenbankspezifischen Dokumentation](https://www.sqlite.org/lang_datefunc.html).

In [15]:
%sql SELECT MIN(Bestelldatum), MAX(Bestelldatum) FROM Bestellung;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


MIN(Bestelldatum),MAX(Bestelldatum)
2017-09-22 08:49:18.787870,2020-06-18 12:02:55.724942


Beachten Sie, dass diese Funktionen nur im Select Teil beheimatet sind, so ist der Ausdruck
```
SELECT * from Bestellung WHERE Bestelldatum = MAX(Bestelldatum);
```
falsch. Probieren Sie es aus.

Was ist nun wenn wir wissen wollen, aus wie vielem Städten unsere Kunden kommen? Probieren wir es mal naiv aus:

In [16]:
%sql SELECT COUNT(Wohnort) FROM Kunde;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


COUNT(Wohnort)
502


das kann nicht stimmen! Und natürlich hat die Datenbank alle unsere Kunden und deren Wohnorte gezählt. Was wir aber wollten waren die Anzahl der unterschiedlichen Städte. Und das wiederrum lößt man in SQL mit dem `DISTINCT` Keywort:

In [17]:
%sql SELECT COUNT(DISTINCT Wohnort) AS Anzahl FROM Kunde;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Anzahl
7


In [18]:
%sql SELECT DISTINCT Wohnort FROM Kunde;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Wohnort
None
San Francisco
London
Landshut
Passau
Deggendorf
Vilshofen
München


### 2.3.2 Stringfunktionen, wie z.B. UPPER, SUBSTR, oder Konkatenation
Sie können die typischen String Operationen aus der Programmiersprache Ihrer Wahl meist in der datenbankspezifischen Dokumentaion finden. Im folgenden Beispiel verwenden wir 
- `UPPER` um den Text in GROßBUCHSTABEN auszugeben
- `SUBSTRING(von,bis)` um einen Teil des Textes zu erhalten 
- `||` um Texte aneinander anzufügen

Darüber hinaus benennen wir die Ergebnisspalte in "Kurzname" um.

In [19]:
%sql SELECT SUBSTR(UPPER(Vorname), 1,1) || '. ' ||  UPPER(Nachname) as Kurzname FROM Kunde LIMIT 3;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kurzname
B. ELSER
W. BERG
J. SCHNEEBERGER


<a name="24">
</a>

## 2.4 Subselects (Unterabfragen)

Statt einfach eine oder mehrere Tabellen im FROM Teil aufzulisten können Sie noch weitere Formen der Abfrage verwenden. Sie Können z.B. die Ergebnisse einer Abfrage direkt wieder verwenden. Ein sog. **subselect** ist praktisch, wenn Sie z.B. wissen wollen, wer Ihr ältester Kunde ist. Der Erste Schritt geht noch ohne einen Subselect:


### 2.4.1 Einführendes Beispiel: Subselect in der `WHERE` Klausel

In [ ]:
%sql SELECT MIN(Geburtstag) from Kunde;

Was aber nun nicht funktioniert ist den Namen des Kunden rauszufinden: (O.K. es geht doch, das ist eine Eigenart von SQLite ;)

In [ ]:
%sql SELECT Vorname,Nachname,MIN(Geburtstag) FROM Kunde;

In [ ]:
%sql SELECT Vorname,Nachname,Geburtstag FROM Kunde where Geburtstag = (SELECT MIN(Geburtstag) FROM Kunde);

Ein anderes Beispiel, bei dem auch in der `WHERE` Klausel gearbeit wird ist die Frage *Welche Kunden haben noch nie bestellt?* Diese kann mit einem subselect beantwortet werden, dabei lernen wir auch den `IN` respektive `NOT IN` Operator kennen, der auf Mengenzugehörigkeit prüft: 

In [20]:
%%sql
SELECT *
FROM Kunde
WHERE Kundennummer  NOT IN (SELECT Kunde FROM Bestellung)
;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
501,Wendy,Alfaro,Deggendorf,99469,None,1989-07-04
502,Roderick,Kinlaw,Deggendorf,99469,None,1990-08-21
503,Richard,Miller,Vilshofen,91234,None,2009-03-13


### 2.4.2 Subselect in der FROM Klausel

Intuitiv einleuchtend ist es, dass ein Ergebnis einer Abfrage als Menge von Tupeln auch als Relation interpretiert werden kann und somit neben den anderen Relationen in der `FROM` klausel stehen kann:

In [22]:
%%sql
SELECT Nachname, Vorname, Count
FROM Kunde, (
    SELECT 
      Kunde as Kundennummer, COUNT(*) AS Count
    FROM
      Bestellung
    GROUP BY
      Kunde
) AS Anzahlen
WHERE 
   Kunde.Kundennummer = Anzahlen.Kundennummer
  AND
   Count > 1
LIMIT 5;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Nachname,Vorname,Count
Elser,Benedikt,219
Berg,Waldemar,200
Schneeberger,Josef,209
Merriweather,Fred,238
Kinlaw,Juan,199


### 2.4.3 Subselect in der `SELECT` Klausel

In der SELECT Klausel geben wir an, welche Attribute pro Zeile angezeigt werden. Daher ist es nicht verwunderlich, dass ein Subselect an dieser Stelle einen einzelnen Wert liefern muss. Falls Ihr Subselect mehrere Werte zurück gibt können Sie ihn aber immer noch durch eine Aggregatsfunktion jagen:

In [23]:
%%sql
SELECT 
  (Kunde.Vorname || " " || Kunde.Nachname) AS Name, 
  COUNT(*) AS Anzahl_Bestellungen,
  (SELECT COUNT(*) as cnt FROM Bestellung GROUP BY Kunde ORDER BY cnt DESC LIMIT 1) as Max_Bestellungen
FROM Kunde
JOIN Bestellung
WHERE Kundennummer = Kunde
GROUP BY Kundennummer
ORDER BY Anzahl_Bestellungen DESC
LIMIT 5;


   sqlite:///
 * sqlite:///bestellungen.db
Done.


Name,Anzahl_Bestellungen,Max_Bestellungen
Juan Vail,244,244
Gregory Abdelhamid,244,244
Nathan Caplan,243,244
Fred Merriweather,238,244
Gregory Elliot,236,244


### 2.4.4 Der `WITH` Operator

Beim Subselect soll eines nicht unerwähnt bleiben: Sie erhöhen die Lesbarkeit des ohnehin schwierig lesbaren SQL Codes nicht wirklich. Daher gibt es in neueren Versionen des SQL Standards den `WITH` Operator. Mit dem können Sie eine temporäre View definieren, die automatisch gelöscht wird, sobald die Abfrage beendet ist. So können Sie in Ihrer Abfrage einfach den namen der neuen "View" verwenden, erhöhen die LEsbarkeit und müssen sich nicht um das Aufräumen kümmern. Hier ein primitves Beispiel:

In [25]:
%%sql

WITH maxGeb AS (SELECT MAX(Geburtstag) as Geburtstag FROM Kunde)
SELECT * FROM Kunde,maxGeb where Kunde.Geburtstag = maxGeb.Geburtstag;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag,Geburtstag_1
362,Annette,Tanner,Landshut,84036,None,2009-11-17,2009-11-17


`WITH` gibt es auch rekursiv, aber das sprengt den Rahmen.

<a name="3">
</a>

# 3 WHERE: Filtern der Ergebnisse

In der WHERE Klausel steckt ein Großteil der Macht einer Datenbank. Hier können Sie jedes Attribut in den angefragten Tabellen prüfen und mehere prüfungen logisch zu komplexen Abrfragen verknüpfen.

<a name="31">
</a>

## 3.1 Logische Operationen
- AND: Logische Und-Verknüpfung
- OR: Logische Oder-Verknüpfung
- NOT: Invertierung des Warheitswerts

Beispiel: Nur Benedikt Elser oder Josef Schneeberger finden, beide dürfen keinen Geburtstag haben

In [26]:
%%sql
SELECT * from Kunde 
WHERE (
    (Vorname = 'Benedikt' AND Nachname = 'Elser') 
       OR 
    (Vorname = 'Josef' AND Nachname = 'Schneeberger')) 
AND Geburtstag IS NULL

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,None,None,None,None
3,Josef,Schneeberger,London,None,None,None


<a name="32">
</a>

## 3.2 Mengenwertige Suchen
Wenn Sie nach mehreren Werten suchen wollen, die Ihr Attribut annehmen kann z.B. die Nachnamen "Elser" oder "Schneeberger" können Sie den `IN` Operator verwenden:

In [27]:
%sql SELECT * from Kunde WHERE Nachname IN ('Elser', 'Schneeberger');

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,None,None,None,None
3,Josef,Schneeberger,London,None,None,None


Mit Numerischen Daten funktioniert das auch:

In [28]:
%sql SELECT * from Kunde WHERE Kundennummer IN (1,2);

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,None,None,None,None
2,Waldemar,Berg,San Francisco,None,None,None


Suchen Sie nach Werten in einer Spanne von *x* bis *y* verwenden Sie `BETWEEN x AND y`:

In [29]:
%sql SELECT * from Kunde WHERE Kundennummer BETWEEN 11 AND 13;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
11,Alfred,Nease,Vilshofen,91234,None,2000-04-23
12,Isabel,Caouette,Landshut,84036,None,1984-05-04
13,David,Reynolds,Landshut,84036,None,2006-02-26


In [30]:
%sql SELECT * from Kunde WHERE Geburtstag BETWEEN '1983-09-24' AND '1988-01-01' LIMIT 5;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
5,Juan,Kinlaw,Landshut,84036,None,1986-03-23
8,Nathan,Barney,Deggendorf,99469,None,1987-08-26
12,Isabel,Caouette,Landshut,84036,None,1984-05-04
20,Alfred,Eisman,Deggendorf,99469,None,1985-09-06
23,Nicole,Elliot,Passau,90123,None,1985-07-13


<a name="33">
</a>

## 3.3 Suche in Texten
Der `LIKE` Operator, sucht nach ähnlichen Test Mustern. Der Platzhalter für beliebigen Text ist %. **Ein Wort der Vorsicht**: Suchabfragen sind inperformant, warum lernen wir noch. Fürs Erste: Stellen Sie sicher, dass Sie den Beginn des Wortes kennen (`LIKE 'El%'`). Anders herum (`LIKE '%ser'`) wird die Abfrage langsamer.

Zum Beispiel: Welche Kunden haben einen Nachnamen, der mit B beginnt:

In [31]:
%sql SELECT * FROM Kunde WHERE Nachname LIKE 'B%' LIMIT 5;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
2,Waldemar,Berg,San Francisco,None,None,None
8,Nathan,Barney,Deggendorf,99469,None,1987-08-26
17,Nathan,Barney,Vilshofen,91234,None,2005-07-30
22,John,Billingsley,Landshut,84036,None,2004-08-23
24,Isabel,Billingsley,München,81377,None,1992-10-26


<a name="joins">
</a>

# 4. Abfragen über mehrere Tabellen: JOINS

![](https://sqlite.org/images/syntax/join-operator.gif)

Bei der Modellierung waren wir immer bestrebt Redundanzen in der Datenbank zu vermeiden. Wir haben Information auf mehrere Tabellen verteilt. Die Frage ist nun: Wie bekommen wir diese Informationen wieder zusammen? Die Antwort ist: Über explizites oder implizites Zusammenführen von Tabellen, also `JOIN`-en der Daten.

Betrachten wir ein Beispiel: Wir wollen nun in unserem Warenhaus die Daten der Kunden mit Ihren Bestellungen zusammen führen. In der Einleitung zu diesem Kaptiel haben wir angesprochen, dass man in der FROM Klausel mehrere Tabellen angeben kann. Das wollen wir für unser Beispiel einmal testen. Der Übersicht halber schränken wir das Ergebnis auf 4 Resultate ein.

In [32]:
%sql SELECT Vorname,Nachname,Kundennummer,Bestellung.* FROM Kunde,Bestellung LIMIT 4;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Vorname,Nachname,Kundennummer,Bestellnummer,Kunde,Bestelldatum
Benedikt,Elser,1,1,450,2018-05-08 19:28:43.712535
Benedikt,Elser,1,2,117,2018-07-06 15:47:35.712558
Benedikt,Elser,1,3,497,2017-12-28 05:53:13.712568
Benedikt,Elser,1,4,143,2019-06-21 14:40:27.712577


<div class="warn">
Das Ergebebis ist falsch. Sie es daran, dass die <b>Kundennummer</b> in der dritten Spalte und die Spalte <b>Kunde</b> nicht gleich sind. Die Datenbank hat nicht einfach das Richtige gemacht und unsere Gedanken gelesen, sondern das <b>Kreuzprodukt</b> gebildet. Jeder Datensatz ist mit jedem anderen verknüpft worden. 
</div>

"Das Richtige" zu machen hieße Tabellen anhand Ihrer Primary Key / Foreign Key Bezeihung miteinander zu verknüpfen. Oder einfach die gleich benannten Attribute verwenden. Letzere Funktion existiert als `NATURAL JOIN` wird aber nicht überall unterstützt und kann zu unerwarten Nebeneffekten führen, wenn Tabellen zwar gleich benannte Attribute haben, diese aber inhaltlich nicht zusammen passen. Daher sehen wir von dieser Verwendung sicherheitshalber ab und merken uns:  

**Ein SELECT über mehrere Tabellen ist ein impliziter Join. Sie müssen also angeben, welche(s) Attribut(e) einer Tabelle mit dem (oder denen) einer anderen Tabelle korrespondiert(en). Im Normalfall müssen sie die Fremd-Primärschlüsselrelation angeben.**

Um das zu korrigieren müssen wir dem System mitteilen was denn die Attribute sind, die gleich seien müssen. Wir versuchen es erneut:

In [33]:
%%sql 
SELECT 
  Vorname,Nachname,Kundennummer,Bestellung.* 
FROM 
  Kunde,Bestellung 
WHERE 
    Kunde.Kundennummer = Bestellung.Kunde
LIMIT 4;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Vorname,Nachname,Kundennummer,Bestellnummer,Kunde,Bestelldatum
Roderick,Caplan,450,1,450,2018-05-08 19:28:43.712535
Nathan,Caplan,117,2,117,2018-07-06 15:47:35.712558
Nicole,Brazill,497,3,497,2017-12-28 05:53:13.712568
Clara,Ashford,143,4,143,2019-06-21 14:40:27.712577


Der Ausdruck lässt sich als expliziter JOIN schreiben. Falls das Attribut in beiden Tabellen gleich heißt können Sie `USING`, hier also z.B. `USING(Kundennummer)` verwenden. Nachdem die Attribute hier einmal Kundennummer und einmal nur Kunde heißen geben Sie die Join Klausel mit `ON` an, wie im folgenden Beispiel:

In [34]:
%%sql
SELECT 
  Vorname,Nachname,Kunde.Kundennummer, Bestellung.* 
FROM 
  Kunde
JOIN Bestellung 
ON
  Kunde.Kundennummer = Bestellung.Kunde
LIMIT 4;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Vorname,Nachname,Kundennummer,Bestellnummer,Kunde,Bestelldatum
Roderick,Caplan,450,1,450,2018-05-08 19:28:43.712535
Nathan,Caplan,117,2,117,2018-07-06 15:47:35.712558
Nicole,Brazill,497,3,497,2017-12-28 05:53:13.712568
Clara,Ashford,143,4,143,2019-06-21 14:40:27.712577


Was wir nun durchgeführt haben ist ein `INNER JOIN`. Das bedeutet, dass wir **keine** Kunden aufgelistet bekommen, die noch keine Bestellung getätigt haben und umgekehert auch keine Bestellung, zu denen der Kundendatensatz fehlt. Zweiteres sollte sowieso bereits durch die Konsistzenzprüfung der Datenbank gewährleistet sein. Bildlich gesprochen schaut es wie folgt aus:

![Inner Join](https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/SQL_Join_-_07_A_Inner_Join_B.svg/220px-SQL_Join_-_07_A_Inner_Join_B.svg.png)

Das geht natürlich auch mit vier Tabellen, und damit komplettieren wir dieses Beispiel: Lassen Sie uns herausfinden, wer wann was gekauft hat! 

In [35]:
%%sql
SELECT 
  Vorname,Nachname,Bestelldatum,Menge,Name
FROM 
  Kunde JOIN Bestellung 
ON
  Kunde.Kundennummer = Bestellung.Kunde
JOIN
  enthaelt
ON enthaelt.Bestellung = Bestellung.Bestellnummer
JOIN
  Artikel
ON enthaelt.Artikel = Artikel.Artikelnummer
ORDER BY
  Bestellung.Bestelldatum ASC
LIMIT 10;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Vorname,Nachname,Bestelldatum,Menge,Name
David,Wise,2017-09-22 08:49:18.787870,1,"Kearui Spielzeug 3-12 Jahren für mädchen, Walkie Talkies für Kinder 8 Kanal Funkgerät mit Hintergrundbeleuchteter LCD-Taschenlampe, 3 Meilen Reichweite für Abenteuer im Freien, Camping, Wandern"
David,Wise,2017-09-22 08:49:18.787870,3,DJI Mavic Mini Care Refresh
Wendy,Elliot,2017-09-22 08:51:17.038866,3,"LEGO Creator 31058 - Dinosaurier, Dinosaurier Spielzeug"
Wendy,Elliot,2017-09-22 08:51:17.038866,3,4153 Fun Science - Geheimnisvolle Kristallwelt. Verrückte Kristallobjekte selbst züchten. KOSMOS Experimentierset für Einsteiger.
Wendy,Elliot,2017-09-22 08:51:17.038866,4,"PROACC Klavier Playmat, Kinder Klaviertastatur Musik Playmat Spielzeug, große Größe (39 * 14 Zoll) lustige Tanzmatte für Babys Kleinkind Jungen und Mädchen Geschenk"
Wendy,Elliot,2017-09-22 08:51:17.038866,3,LEGO 76897 Speed Champions 1985 Audi Sport Quattro S1 Rennwagenspielzeug
Sara,Ornelas,2017-09-22 09:08:34.851960,2,"EXTSUD Piano Mat Tanzmatten Klaviermatte Musikmatte Kinder 8 Tierstimmen Klaviertastatur Spielzeug Musik Matte, Keyboard Matten Spielteppich Baby Tanzmatte für Jungen Mädchen Kinder 100*36 cm"
Sara,Ornelas,2017-09-22 09:08:34.851960,4,"Twister, Geschicklichkeitsspiel für Kinder & Erwachsene, Familienspiel, Partyspiel, lustiges Spiel für Kindergeburtstage, 2-4 Personen, ab 6 Jahren"
Sara,Ornelas,2017-09-22 09:08:34.851960,4,Spin Master Games 6058355 OGM Perplexus 2x2 Rubiks GML
Nathan,Caplan,2017-09-22 09:29:41.960481,2,"LEGO 75979 Harry Potter Hedwig, Bauset, Mehrfarbig"


<a name="41">
</a>
    
## 4.1 Die verschiedenen Joins

Es gibt verschiedene Arten Joins zu erklären. Ich bevorzuge immer noch folgendes Bild:

<img src="https://i.stack.imgur.com/4zjxm.png" width="60%" />

Es gibt aber auch Vetreter der Zunft die [diese Erklärung besser finden](https://blog.jooq.org/2016/07/05/say-no-to-venn-diagrams-when-explaining-joins/)

Um uns die verschiedenen weiteren JOIN Arten anzusehen, legen wir kurz unser Warenhaus zur Seite und verwenden einfachere Beispieltabellen: Sie heißen ganz einfach "eins" und "zwei". Anhand derer wollen wir die verschiedenen Arten der Verknüpfung durchspielen:

![](https://nextcloud.th-deg.de/s/ctmxwHMqK3Ybpi5/download)

In [36]:
%%sql
CREATE TABLE eins(id INT PRIMARY KEY, val CHAR(1));
INSERT INTO eins VALUES(1,'A'), (2,'B'), (3,'C');
CREATE TABLE zwei(id INT PRIMARY KEY, val2 CHAR(1));
INSERT INTO zwei VALUES(2,'D'), (3,'E'), (4,'F');

   sqlite:///
 * sqlite:///bestellungen.db
Done.
3 rows affected.
Done.
3 rows affected.


[]

### Aufgabe
Führen sie an den Tabellen die [verschiedenen JOIN Arten](https://nextcloud.th-deg.de/s/oS6sirZ8PXRSAe7/download) durch, implementieren Sie
<ul>
    <li>LEFT JOIN</li>
    <li>RIGHT JOIN</li>
    <li>A - B</li>
    <li>B - A</li>
</ul>
um ein Gefühl für die veränderte Ergebnissmenge zu erhalten.


In [37]:
%%sql
SELECT *
FROM eins
JOIN zwei
USING(id);

   sqlite:///
 * sqlite:///bestellungen.db
Done.


id,val,val2
2,B,D
3,C,E


<a name="5">
</a>

# 5. Mehr SQL

<a name="51">
</a>

## 5.1. Sortieren und Limitieren

#### ORDER BY <Attribut> [ASC | DESC]:
Sortiert nach einem oder mehreren Attributen auf- oder absteigend.

#### LIMIT:
Oft sieht man den `*` als Platzhalter für **alle** Attribute einer Tabelle. Im nächsten Aufruf lernen wir auch eine Möglichkeit kennen zwar alle Attribute, aber nur die erste Zeile der Ergebnisse zu erhalten. Den `LIMIT n` Befehlt, der genau *n* Zeilen des Ergebis ausgibt.

In [38]:
%%sql
SELECT * from Kunde LIMIT 1;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,None,None,None,None


<div class="aufgabe"> Lassen Sie den LIMIT Befehl doch einmal weg und lassen Sie sich alle Kunden ausgeben!</div>

<a name="52">
</a>

## 5.2 Gruppierung

Es ist nicht unüblich, dass Sie Daten aggregiert betrachten wollen, anstatt das bewährte "eyballing" zu verwenden. Es gibt aber etwas zu bedenken:
Wenn Sie Gruppen von Daten verwenden, haben Sie ja nicht mehr einzelne Tupel, die Sie verarbeiten wollen, sondern Tupelmengen. Wenn Sie auf einzeltupelebene bleiben wollen würden könnten Sie diese ja sonst einfach entsprechend sortieren. Und aber eine Menge in einer Zeile anzuzeigen muss man diese auf einen einzelnen Wert reduzieren. Also z.B. die Summe bilden mit `sum()`, Minimum und Maximum bilden mit `min() max()`, oder einfach alles als String hintereinander hängen mit `group_concat()`.

Betrachten wir ein Beispiel. Als erstes sortieren wir einfach mal:

In [39]:
%sql SELECT * FROM Kunde WHERE Kundennummer < 10 ORDER BY Wohnort;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Kundennummer,Vorname,Nachname,Wohnort,PLZ,Adresse,Geburtstag
1,Benedikt,Elser,None,None,None,None
8,Nathan,Barney,Deggendorf,99469,None,1987-08-26
4,Fred,Merriweather,Landshut,84036,None,1983-06-12
5,Juan,Kinlaw,Landshut,84036,None,1986-03-23
6,John,Middleton,Landshut,84036,None,1998-01-03
3,Josef,Schneeberger,London,None,None,None
7,Annette,Marshall,Passau,90123,None,1989-08-03
9,Annette,Caplan,Passau,90123,None,1990-08-27
2,Waldemar,Berg,San Francisco,None,None,None


Nun kann man sich jeden Datensatz einzeln anschauen, aber wir wollen ja zu Gruppen zusammenfassen, also eine Zeile für Landshut, eine für Passau, etc. In Passau wohnen aber im obigen Datensatz zwei Kunden. Wie soll die Datenbank wissen, welchen Kunden sie Anzeigen soll (und macht das überhaupt Sinn?). Wir müssen also eine Aggregatsfunktion verwenden, die aus vielen Datensätzen einen macht. Hier bilden wir die Summe und zählen gleichzeitig auch noch wie viele Kunden das sind:

In [41]:
%sql SELECT group_concat(Nachname) AS Einwohner, count(*) as Anzahl, Wohnort FROM Kunde WHERE Kundennummer < 11 GROUP BY Wohnort;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Einwohner,Anzahl,Wohnort
Elser,1,None
Barney,1,Deggendorf
"Merriweather,Kinlaw,Middleton,Abdelhamid",4,Landshut
Schneeberger,1,London
"Marshall,Caplan",2,Passau
Berg,1,San Francisco


#### HAVING vs. WHERE bei GROUP BY

Wenn wir nun diese Menge weiter einschränken wollen funktioniert unser bewährtes `WHERE` nicht mehr. Dieses arbeitet **vor** der Aggregation. Falls wir nun aber nur Städte anzeigen wollen, die mehr als einen Kunden beheimaten verwenden wir den `HAVING`Operator. Dieser wird nach der Aggregation aufgerufen:

In [42]:
%sql SELECT group_concat(Nachname) AS Einwohner, count(*) as Anzahl, Wohnort FROM Kunde WHERE Kundennummer < 11 GROUP BY Wohnort HAVING Anzahl > 1;

   sqlite:///
 * sqlite:///bestellungen.db
Done.


Einwohner,Anzahl,Wohnort
"Merriweather,Kinlaw,Middleton,Abdelhamid",4,Landshut
"Marshall,Caplan",2,Passau


<a name="53">
</a>

## 5.3 Ein Transaktions-Beispiel

Leider unterstützt unsere Umgebung keine Transaktionen. Wenn Sie diesen Code jedoch in einer Datenbank Shell ausführen sehen Sie das Wunder von Transaktionen: Im Beispiel versuchen wir eine Überweisung von Kunden C zu Kunden B zu machen. Leider ist das Konto von C nicht mehr gedeckt und die Abbuchung schlägt fehl. Weil wir aber eine Transaktion als "umgebende Klammer" mit `BEGIN TRANSACTION` und `COMMIT` gewählt haben dürfen wir erwarten, dass nach Ausführung der Transaktion B immer noch "nur" 2000 Geldeinheiten, und C immer noch 90 Geldeinheiten hat. Sehen Sie sich die Situation vor dem Abbuchen und nach dem Aufbuchen an, sieht das zunächst noch anders aus.

```
.headers on
.mode column
DROP TABLE IF EXISTS Konten;
CREATE TABLE IF NOT EXISTS Konten(kunde text PRIMARY KEY, balance integer, CHECK (balance >= 0));
INSERT INTO Konten(kunde,balance) VALUES('A', 1000), ('B', 2000), ('C', 90);
```

```
BEGIN TRANSACTION;

UPDATE OR ROLLBACK Konten SET balance = balance + 100 WHERE kunde = 'B';
UPDATE  OR ROLLBACK Konten SET balance = balance - 100 WHERE kunde = 'C';

COMMIT;
```